In [598]:
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# This is a swing at a custom neural network predicting game results based on scores.
# I've been in Keras for a few months, but am just starting to understand my way around.
df = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv").dropna()

# We need to calculate one team's aggregate margin in order to determine the result.
# We've chosen home for that calculation, but may as well get away as well for good measure.
# Then, the absolute margin is the number of goals between the teams.
df['hm'] = df['score1'] - df['score2']
df['am'] = df['score2'] - df['score1']
df['margin'] = np.abs(df['score1'] - df['score2'])

results = []

for i in df['hm']:
  if i > 0: # If the home team's margin is greater than 0, it's a home win.
    results.append("HOME WIN")
  elif i < 0: # If the home team's margin is less than 0, it's an away win.
    results.append("AWAY WIN")
  else: # Otherwise, it's a draw.
    results.append("DRAW")
    
df['result'] = results

In [599]:
# The model gets a little confused the more numerical rows are fed into it, so we're keeping it simple.
X = df[['score1', 'score2', 'hm', 'am', 'margin']].reset_index(drop=True)
y = df[['result']].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=6)

In [600]:
# Have to factorize the labels before converting them to categorical (e.g. 1.,0.,0.).
y_train = to_categorical(pd.factorize(y_train['result'])[0])
y_test = to_categorical(pd.factorize(y_test['result'])[0])

# The train and test data will be easier to handle in array form.
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)

In [601]:
# Three layers with a 3 unit output layer given the number of outcomes (HOME WIN, AWAY WIN, DRAW).
model = Sequential()
model.add(Dense(1024, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [602]:
# Definitely excess on the epochs. ¯\_(ツ)_/¯
model.fit(X_train, y_train, epochs=40, batch_size=128, validation_split=0.2, verbose=1)

Epoch 1/40
75/75 [==============================] - 1s 11ms/step - loss: 0.1995 - accuracy: 0.9721 - val_loss: 6.1976e-04 - val_accuracy: 1.0000
Epoch 2/40
75/75 [==============================] - 1s 8ms/step - loss: 2.6382e-04 - accuracy: 1.0000 - val_loss: 5.0535e-06 - val_accuracy: 1.0000
Epoch 3/40
75/75 [==============================] - 1s 8ms/step - loss: 2.6054e-06 - accuracy: 1.0000 - val_loss: 1.5270e-07 - val_accuracy: 1.0000
Epoch 4/40
75/75 [==============================] - 1s 8ms/step - loss: 7.3994e-08 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/40
75/75 [==============================] - 1s 8ms/step - loss: 7.6786e-11 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/40
75/75 [==============================] - 1s 8ms/step - loss: 9.9552e-11 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/40
75/75 [==============================] - 1s 9ms/step - loss: 7.1477e-11 - accuracy: 1.0000 - val_loss: 0

In [603]:
# Now we will predict results based on test data.
predictions = model.predict(X_test).round(2)

In [604]:
# Reshaping y_test and predictions to be directly compared 1v1 iteratively.
predictions = predictions.reshape(23814,)
predictions = pd.Series(predictions)
predictions = pd.DataFrame(predictions).reset_index(drop=True)
y_test = y_test.reshape(23814,)
y_test = pd.Series(y_test)
y_test = pd.DataFrame(y_test).reset_index(drop=True)
PvA = pd.concat([y_test, predictions], axis=1)
PvA.columns = ["Predicted", "Actual"]

In [605]:
# If the margin is 0, there is no error. If the margin is 1, there was an error.
PvA['Margin'] = np.abs(PvA['Predicted'] - PvA['Actual'])

In [607]:
# Sum of all of the rows with errors.
PvA_err = np.sum(PvA['Margin'])

In [608]:
# Length of the prediction set.
PvA_len = len(PvA)

In [609]:
# Accuracy metric.
PvA_acc = 1-(PvA_err/PvA_len)

In [610]:
# AORTD = Accuracy on Random Test Data
print("AORTD: %.2f" % (PvA_acc * 100) + "%")

AORTD: 63.69%
